If a class define getattr, that method is called every time an attribute can't be found in an object's instance dictioary

In [1]:
class LazyRecord:
    def __init__(self):
        self.exists = 5
        
    def __getattr__(self, name):
        value = f'Value for {name}'
        setattr(self, name, value)
        return value

In [2]:
data = LazyRecord()
print('Before:', data.__dict__)
print('foo:   ', data.foo)
print('After: ',data.__dict__)

Before: {'exists': 5}
foo:    Value for foo
After:  {'exists': 5, 'foo': 'Value for foo'}


In [3]:
class LoggingLazyRecord(LazyRecord):
    def __getattr__(self, name):
        print(f'* Called __getattr__({name!r}), '
              f'populating instance dictionary')
        result = super().__getattr__(name)
        print(f'* Returning {result!r}')
        return result
    
data = LoggingLazyRecord()
print('exists:     ', data.exists)
print('First foo:  ', data.foo)
print('Second foo: ', data.foo)

exists:      5
* Called __getattr__('foo'), populating instance dictionary
* Returning 'Value for foo'
First foo:   Value for foo
Second foo:  Value for foo


In [4]:
class ValidatingRecord:
    def __init__(self):
        self.exists = 5
    
    def __getattribute__(self, name):
        print(f'* Called __getattribute__({name!r})')
        try:
            value = super().__getattribute__(name)
            print(f'* Found {name!r}, returning {value!r}')
            return value
        except AttributeError:
            value = f'Value for {name}'
            print(f'* Setting {name!r} to {value!r}')
            setattr(self, name, value)
            return value
        

data = ValidatingRecord()
print('exists:     ', data.exists)
print('First foo:  ', data.foo)
print('Second foo: ', data.foo)        

* Called __getattribute__('exists')
* Found 'exists', returning 5
exists:      5
* Called __getattribute__('foo')
* Setting 'foo' to 'Value for foo'
First foo:   Value for foo
* Called __getattribute__('foo')
* Found 'foo', returning 'Value for foo'
Second foo:  Value for foo


In [5]:
#In the event that a dynamically accessed property shouldn't exist, 
#I can raise an AttributeError to cause Python standard missing

class MissingPropertyRecord:
    def __getattr_(self, name):
        if name == 'bad_name':
            raise AttributeError(f'{name} is missing')
        pass
    
data = MissingPropertyRecord()
data.bad_name

AttributeError: 'MissingPropertyRecord' object has no attribute 'bad_name'

In [6]:
#rely on the hasattr built-in function to determine when property exist
#hasattr call after __getattr__

data = LoggingLazyRecord()
print('Before:         ', data.__dict__)
print('Has first foo:  ', hasattr(data, 'foo'))
print('After:          ', data.__dict__)
print('Has second foo: ', hasattr(data, 'foo'))

Before:          {'exists': 5}
* Called __getattr__('foo'), populating instance dictionary
* Returning 'Value for foo'
Has first foo:   True
After:           {'exists': 5, 'foo': 'Value for foo'}
Has second foo:  True


In [7]:
#__getattr__ call only once(when attr not exist)
#__getattribute__ call everytime when the attr been called

data = ValidatingRecord() # Implements __getattribute__
print('Has first foo:  ', hasattr(data, 'foo'))
print('Has second foo: ', hasattr(data, 'foo'))

* Called __getattribute__('foo')
* Setting 'foo' to 'Value for foo'
Has first foo:   True
* Called __getattribute__('foo')
* Found 'foo', returning 'Value for foo'
Has second foo:  True


In [8]:
#the __setattr__ method is always called every time na attribute is assigned on an instance(
#either directly or through the setattr built-in function)

class SavingRecord:
    def __setattr__(self, name, value):
        # Save some data for the record
 
        super().__setattr__(name, value)
        
class LoggingSavingRecord(SavingRecord):
    def __setattr__(self, name, value):
        print(f'* Called __setattr__({name!r}, {value!r})')
        super().__setattr__(name, value)

In [9]:
data = LoggingSavingRecord()
print('Before: ', data.__dict__)
data.foo = 5
print('After:  ', data.__dict__)
data.foo = 7
print('Finally:', data.__dict__)

Before:  {}
* Called __setattr__('foo', 5)
After:   {'foo': 5}
* Called __setattr__('foo', 7)
Finally: {'foo': 7}


In [ ]:
class BrokenDictionaryRecord:
    def __init__(self, data):
        self._data = data
    
    def __getattribute__(self, name):
        print(f'* Called __getattribute__({{name!r}})')
        return self._data[name]
    
data = BrokenDictionaryRecord({'foo': 3})
data.foo

In [2]:
#The solution to recurse is to use the super().__getattribute__ to fetch value form
#the instance attribute dictionary
class DictionaryRecord:
    def __init__(self, data):
        self._data = data
    
    def __getattribute__(self, name):
        print(f'* Called __getattribute__({{name!r}})')
        data_dict = super().__getattribute__('_data')
        return data_dict[name]
    
data = DictionaryRecord({'foo': 3})
print('foo: ', data.foo)

* Called __getattribute__({name!r})
foo:  3
